# Lesson 15 - Advanced construction and small types

This notebook supports the materials in [lesson 15 of C++ for Finance](https://loz-hurst.github.io/cpp-finance-materials-new/lessons/lesson-15.html).

## Constructors

Recall that constructors can be called explicitly or implicitly:

In [ ]:
class MyFirstClass {
public:
    MyFirstClass(const int a) {}
};

In [ ]:
MyFirstClass my_first_class_instance_1 {2}; // Explicit call

In [ ]:
MyFirstClass my_first_class_instance_2 = 2; // Implicit call

We can prevent implicit conversion by adding the keyword `explicit` to the constructor:

In [ ]:
class MySecondClass {
public:
    explicit MySecondClass(const int a) {}
};

In [ ]:
MySecondClass my_second_class_instance_1 {2}; // Explicit call - fine

In [ ]:
MySecondClass my_second_class_instance_2 = 2; // Implicit call - fail

### Overloading constructors

We can overload constructors, just like functions:

In [ ]:
#include <string>
#include <iostream>

In [ ]:
class OverloadedConstructors {
public:
    OverloadedConstructors(const int a) {std::cout << "int constructor called" << std::endl;}
    OverloadedConstructors(const int a, const int b) {std::cout << "two ints constructor called" << std::endl;}
    OverloadedConstructors(const double a) {std::cout << "double constructor called" << std::endl;}
    OverloadedConstructors(const std::string & a) {std::cout << "string constructor called" << std::endl;}
};

In [ ]:
OverloadedConstructors overloaded_constructors_instance_1 {2};
OverloadedConstructors overloaded_constructors_instance_2 {2, 4};
OverloadedConstructors overloaded_constructors_instance_3 {2.4};
OverloadedConstructors overloaded_constructors_instance_4 {"two"};

### Default constructor

The default constructor is the no-argument constructor.

In [ ]:
class DefaultConstructor {
public:
    DefaultConstructor() {std::cout << "Default constructor called" << std::endl;}
}

In [ ]:
DefaultConstructor default_constructor_instance_1; // Calls the default constructor

If we do not specify one, C++ will provide a default (empty) one:

In [ ]:
class ImplicitDefaultConstructor {};

In [ ]:
// Fine, uses the C++ provided default
ImplicitDefaultConstructor implicit_default_constructor_instance_1;

However if we provide **any** of our own constructors then C++ will not provide a default one:

In [ ]:
class CustomConstructor {
public:
    CustomConstructor(const int a) {}
}

In [ ]:
CustomConstructor custom_constructor_instance_1; // Invalid - there is no default constructor

But we can get the C++ provided one back it back by explicitly setting it to default:

In [ ]:
class CustomConstructorWithDefault {
public:
    CustomConstructorWithDefault(const int a) {}
    CustomConstructorWithDefault() = default;
};

In [ ]:
// Fine, we now have a default constructor
CustomConstructorWithDefault custom_constructor_with_default_instance_1;

### Deleting constructors

Very occasionally we might want to prevent a class from being instantiated, for example a purely static class (a namespace would be better, unless using templates!).  To do this we need to explicitly say we do not want the C++ provided constructor (as we do not want any constructor), which we can do by setting it to `delete`:

In [ ]:
class DeletedConstructor {
private:
    static const int a {10};
public:
    DeletedConstructor() = delete;
    static void DoSomething() {std::cout << "can still access static members: " << a << std::endl;}
}

In [ ]:
DeletedConstructor deleted_constructor_instance_1; // Error - no constructor

In [ ]:
DeletedConstructor::DoSomething(); // Can still access static members

### Delegating constructors

We have already seen how to call a base class's constructor using the initialiser list.  We can also use this to call another constructor within the same class:

In [ ]:
class DelegatingConstructors {
public:
    DelegatingConstructors(const int a, const int b) {}
    // This constructor delegates to the two value version
    DelegatingConstructors(const int value) : DelegatingConstructors(value, value) {}
};

### Special member functions

Discuss what they including `default`, `delete`, and rule of 5 (3) and zero.

Move constructor/operator and destructor must not throw.

### Copy constructor

In [ ]:
class CopyConstructor {
public:
    CopyConstructor() = default;
    CopyConstructor(const CopyConstructor & other) {std::cout << "Copy constructor called" << std::endl;}
};

In [ ]:
CopyConstructor copy_constructor_instance_1;

In [ ]:
CopyConstructor copy_constructor_instance_2 {copy_constructor_instance_1};

In [ ]:
// Note by-value passing
void MyCopyConstructorFunction(CopyConstructor copy_constructor) {
    std::cout << "Called the function" << std::endl;
}

In [ ]:
MyCopyConstructorFunction(copy_constructor_instance_2);

### Temporary objects

In [ ]:
int a {0}, b{1}, c{2};
a = b + c;
a = b; a += c; // Same thing, more efficient, less intuative

In [ ]:
class TemporaryObjectExample {
public:
    TemporaryObjectExample() = default;
    TemporaryObjectExample(const TemporaryObjectExample& other) : i_{0} {std::cout << "Copy constructor called" << std::endl;}
};

In [ ]:
TemporaryObjectExample MyTemporaryObjectFunction1() {
    return TemporaryObjectExample();
}

In [ ]:
TemporaryObjectExample temporary_objects_example_instance_1 {MyTemporaryObjectFunction1()};

### Converstion operators

We already know that single argument constructors are called "conversion constructors" and, if not explicit, will automatically convert other types to our classes.  We can also specify converstions from our class to other types using converstion operator methods:  (n.b. do not provide both if converting between two types - the compiler will not know which to use!)

In [ ]:
class MyConverstion1 {
public:
    MyConverstion1(const int a) {} // Convert from an int
    operator int() {return 4;} // Convert to an int
};

In [ ]:
class MyConverstion2 {
public:
    MyConverstion2(const int a) {} // Convert from an int
    explicit operator int() {return 3;} // Convert to an int
};

In [ ]:
MyConverstion1 my_conversion_instance_1 {2}; // Explicit conversion rfom int with MyConversion1
MyConverstion2 my_conversion_instance_2 = 3; // Implicit conversion from int with MyConversion2

In [ ]:
int my_conversion_int_1 {my_conversion_instance_1}; // explicit conversion to int
int my_conversion_int_2 = my_conversion_instance_1; // Implicit conversion to int
std::cout << my_conversion_int_1 << std::endl
          << my_conversion_int_2 << std::endl;

In [ ]:
int my_conversion_int_3 {my_conversion_instance_2}; // explicit conversion to int - fine
std::cout << my_conversion_int_3 << std::endl;

In [ ]:
int my_conversion_int_4 = my_conversion_instance_2; // Fail - implicit conversion to int

## Small types

### `std::pair`

We met `std::pair` when looking at maps.  It is in the `utility` header:

In [ ]:
#include <utility>

In [ ]:
std::pair<int, std::string> my_pair {2, "Hi there!"};

In [ ]:
std::cout << my_pair.first << ", " << my_pair.second << std::endl;

### `std::tuple`

These are a generalised version of pair that can hold may (different typed) values.  It is in the `tuple` header:

In [ ]:
#include <tuple>

In [ ]:
std::tuple<int, std::string, double, int> my_tuple {7, "Foo Bar", 2.3, 1009};

In [ ]:
std::tuple<int, std::string, double, int> my_other_tuple =
    std::make_tuple<int, std::string, double, int>(8, "banana", 0.3, -1);

In [ ]:
std::cout << std::get<0>(my_tuple) << std::endl; // Use std::get for member access

In [ ]:
// Can get by type but only if there is just 1 of that type
std::cout << std::get<double>(my_tuple) << std::endl;

In [ ]:
std::cout << std::get<int>(my_tuple) << std::endl; // Error - 2 ints in tuple

In [ ]:
int unpacked_int_a{0}, unpacked_int_b{0};
std::string unpacked_str;
double unpacked_dbl {0.0};
// We can used std::tie to unpack a typle into variables
std::tie(unpacked_int_a, unpacked_str, unpacked_dbl, unpacked_int_b) = my_tuple;
std::cout << unpacked_int_a << ", "
          << unpacked_str << ", "
          << unpacked_dbl << ", "
          << unpacked_int_b << std::endl;

In [ ]:
int unpacked_int_c{0}, unpacked_int_d{0};
// Use std::ignore if we only one some values from the tuple
std::tie(unpacked_int_c, std::ignore, std::ignore, unpacked_int_d) = my_tuple;
std::cout << unpacked_int_c << ", "
          << unpacked_int_d << std::endl;

### `std::array`

`std::array` is a very thin shim around C arrays but, crucially, it knows its own size (as well as having usual C++ container methods) which C arrays do not.  It can only be used if the size is known at compile time.

In [ ]:
std::array<int, 5> my_array; // New array of 5 integers

In [ ]:
void UsesArray(const int size) {
    std::array<int, size> some_array; // Error, size not known at compiler time
}

In [ ]:
std::cout << my_array.size() << std::endl;